In [ ]:
from act.SyntheticGenerator import SyntheticGenerator
from act.act_types import SimulationParameters, OptimizationParameters, CurrentInjection, FilterParameters
from act.cell_model import TargetCell
from act.module_parameters import ModuleParameters

random_seed = 42
module_name = f"LA_A_seg_{random_seed}_module_1"

target_cell = TargetCell(
    path_to_hoc_file="../../data/LAA/orig_modfiles/template.hoc",
    path_to_mod_files="../../data/LAA/orig_modfiles",
    cell_name="Cell_A",
    active_channels=[]
)

sim_par= SimulationParameters(
            h_v_init=-65.0,  # Consistent with nrngui settings
            h_tstop=500,     # Simulation time in ms
            h_dt=0.1,      # Time step in ms
            h_celsius=6.3,   # Temperature in degrees Celsius
            CI=[CurrentInjection(type="constant",amp=0.1,dur=300,delay=100),
                CurrentInjection(type="constant",amp=0.2,dur=300,delay=100),
                CurrentInjection(type="constant",amp=0.3,dur=300,delay=100)]
        )

sg = SyntheticGenerator(
    ModuleParameters(
        module_folder_name=module_name,
        cell= target_cell,
        sim_params= sim_par
    )
)

sg.generate_synthetic_target_data("target_data.csv")

In [ ]:
passive_sim_par = SimulationParameters(
            h_v_init = -65,
            h_tstop = 500,
            h_dt = 0.1,
            h_celsius = 6.3,
            CI=[CurrentInjection(type="constant",amp=-0.1,dur=300,delay=100)]
        )

sg_passive = SyntheticGenerator(
    ModuleParameters(
        module_folder_name=module_name,
        cell= target_cell,
        sim_params= passive_sim_par,
        optim_params = OptimizationParameters(
            blocked_channels= []
        )
    )
)

sg_passive.generate_synthetic_target_data("passive_data.csv")

In [ ]:
from act.PassivePropertiesModule import PassivePropertiesModule
from act.act_types import SimParams
from act.cell_model import TrainCell

train_cell = TrainCell(
    hoc_file="/home/mwsrgf/proj/ACT/data/Spiker/seg/template.hoc",
    mod_folder="/home/mwsrgf/proj/ACT/data/Spiker/seg",
    cell_name="Simple_Spiker_seg",
    g_names = ["gnabar_hh_seg", "gkbar_hh_seg", "gl_hh_seg"]
)

passive_mod = PassivePropertiesModule(
    train_cell=train_cell,
    sim_params=passive_sim_par,
    trace_filepath=f"./model/{module_name}/target/passive_data.csv",
    leak_conductance_variable="gl_hh_seg",
    leak_reversal_variable="el_hh_seg"
)

In [ ]:
passive_mod.set_passive_properties()
print(train_cell.passive_properties)

In [ ]:
from act.ACTModule import ACTModule
from act.SyntheticGenerator import SyntheticGenerator
from act.act_types import  Cell, SimParams, OptimizationParameters, OptimizationParam
from act.cell_model import ModuleParameters
from act.PassivePropertiesModule import PassivePropertiesModule
from act.act_types import SimParams
from act.cell_model import TrainCell

mod = ACTModule(
    ModuleParameters(
        module_folder_name=module_name,
        cell= train_cell,
        target_traces_file = "target_data.csv",
        sim_params= sim_par,
        optim_params= OptimizationParameters(
            g_ranges_slices= [
                OptimizationParam(param="gnabar_hh_seg", low=0.06, high=0.18, n_slices=15),
                OptimizationParam(param="gkbar_hh_seg", low=0.018, high=0.054, n_slices=15),
                OptimizationParam(param="gl_hh_seg", prediction=train_cell.passive_properties.g_bar_leak, bounds_variation=0.0, n_slices=1),
                
            ],
            prediction_eval_method='fi_curve',
            random_state=random_seed
        )
    )
)

In [ ]:
predicted_g_data_file = mod.run()

In [ ]:
mod.pickle_rf(mod.rf_model,f"./model/{module_name}/trained_rf.pkl")

In [ ]:
from act.ACTModule import ACTModule
from act.SyntheticGenerator import SyntheticGenerator
from act.act_types import  Cell, SimParams, OptimizationParameters, OptimizationParam
from act.cell_model import ModuleParameters
from act.PassivePropertiesModule import PassivePropertiesModule
from act.act_types import SimParams
from act.cell_model import TrainCell

mod = ACTModule(
    ModuleParameters(
        module_folder_name=module_name,
        cell= train_cell,
        target_traces_file = "target_data.csv",
        sim_params= sim_par,
        optim_params= OptimizationParameters(
            g_ranges_slices= [
                OptimizationParam(param="gnabar_hh_seg", low=0.06, high=0.18, n_slices=15),
                OptimizationParam(param="gkbar_hh_seg", low=0.018, high=0.054, n_slices=15),
                OptimizationParam(param="gl_hh_seg", prediction=train_cell.passive_properties.g_bar_leak, bounds_variation=0.0, n_slices=1),
                
            ],
            prediction_eval_method='fi_curve',
            random_state=random_seed
        )
    )
)

In [ ]:
predicted_g_data_file = mod.run()

In [ ]:
mod.pickle_rf(mod.rf_model,f"./model/{module_name}/trained_rf.pkl")

In [ ]:
from act import ACTPlot
ACTPlot.plot_v_comparison(predicted_g_data_file, f"./model/{module_name}/target", f"./model/{module_name}", sim_par["CI_amps"])
ACTPlot.plot_fi_comparison(f"./model/{module_name}/results/frequency_data.npy", sim_par["CI_amps"])